In [ ]:
import os
import threading

多线程主要要学会threading.Thread(target,args)方法的运用

In [ ]:
def task(a, b):
    pass

# 创建线程池
threading_pool = []
a = 1
b = 2
for i in range(10):
    thread = threading.Thread(target=task, args=(a, b))  #创建线程
    threading_pool.append(thread)  # 加入线程池
    thread.start()  # 线程准备就绪，等待CPU调度

for thread in threading_pool:
    thread.join()  # 等待主线程执行完毕，再继续运行下面的代码

上面的方法在第一个for循环中，用到了range,这个就可以对线程数进行控制
同时在上面的基础上，也可以再对args的参数进行控制，在外层加一个针对args的循环，让每个线程对遍历一遍args；
或者在内层加一个针对args的for循环，针对每个args均创建range个线程

# 创建线程的第二种方法，通过类创建
除了上面直接通过threading.Thread(target,args)方法创建外，可以写一个类继承threading.Thread类，然后重写__init__()和run()方法，这样就可以在创建线程时传入参数；
这里如果通过重写__init__()方法，可以把外部方法引入这个类中，类似运用threading.Thread(target,args)
如果新加一个类中的方法如run(),就可以对这个线程的创建进一步定制化

In [ ]:
class MyThread(threading.Thread):
    def __init__(self, target, args):
        super().__init__(target=target, args=args)

    def run(self):
        pass


t = MyThread(target=task, args=(a, b))
t.start()

In [ ]:
thread = MyThread(target=task, args=(a, b))

# 守护线程(要在start()之前)

In [ ]:
thread2 = threading.Thread(target=task, args=(a, b))
thread2.setDaemon(True)  # 守护线程，主线程执行完毕，所有子线程随之全部结束
thread2.setDaemon(False)  # 非守护线程，所有子线程执行完毕后，主线程才结束

# 设置线程名称(要在start()之前)
可以为每个线程单独命名，有利于后面对线程更详细的控制

In [ ]:
thread.setName(f'thread-{a}')

# 线程安全
如果多线程执行2个方法，两个方法就会互相干涉，造成结果出错，这就是线程不安全
为了解决这个问题，就需要引入线程锁，让两个方法运用同一把锁。使用线程时需要先申请锁，用完再释放锁，这样就能保证不会同时干涉

In [ ]:
import threading

lock = threading.RLock()  # 创建锁


def task_1(a, b):
    lock.acquire()  # 申请锁
    # do something
    lock.release()  # 释放锁


def task_2(a, b):
    lock.acquire()
    # do something
    lock.release()


t1 = threading.Thread(target=task_1, args=(a, b))
t2 = threading.Thread(target=task_2, args=(a, b))
t1.start()
t2.start()
t1.join()
t2.join()

上面演示了在方法中手动申请锁（加锁）和释放锁（解锁），但其实更常用的是with上下文管理器实现自动加锁/解锁

In [ ]:
def task_3(a, b):
    with lock:
        # do something
        print("lock operate auto")

在python的开发工作中，有些操作本身是集成锁的，也就是自带线程安全，具体要看开发文档举例，list.append()方法就是线程安全的，实际上针对list和dict对象的大多数操作如sort，赋值，取值等等，都是线程安全的。还是一句话：
-- 要看开发文档

threading.Lock()和threading.RLock()
### Lock()是互斥锁。仅支持加锁-解锁，不支持多次加锁-多次解锁的嵌套锁
### RLock()是递归锁。不仅支持加锁-解锁，也支持多次加锁-多次解锁的嵌套锁

In [ ]:
import threading

lock2 = threading.Lock()  # 创建单锁
lock3 = threading.RLock() # 创建嵌套锁

def task_1(feature):
    lock2.acquire()  # 申请锁
    # do something
    print(f'{feature.result()}')
    lock2.release()  # 释放锁


def task_2(a, b):
    lock3.acquire()  # 申请锁1
    lock3.acquire()  # 申请锁2
    lock3.acquire()  # 申请锁3
    # do something
    return a+b
    lock3.release()  # 释放锁3
    lock3.release()  # 释放锁2
    lock3.release()  # 释放锁1


# 最常见RLock写法
def task_3(a,b):
    with lock3:
        with lock3:
            with lock3:
                # do something
                print("Rlock operate auto")

### 为什么要推荐用RLock，因为协同开发时，可能会出现不同开发人员多次加锁的情况，Lock并不支持这么做，会造成死锁

# 死锁
-1 出现只有加锁，没有解锁的情况，当再次加锁时，就会造成死锁
-2 用Lock()进行嵌套加锁时，会造成死锁

# 线程池
线程池的作用：限制创建有限个线程，即线程池是有容量限制的。因为线程无限增多后，其实对性能的损失是非常大的。

In [ ]:
from concurrent.futures import ThreadPoolExecutor

pool = ThreadPoolExecutor(100) # 创建一个线程池，容量限制100

# 线程池的用法

In [ ]:
futures = pool.submit(task_3,a,b) # 将task_3方法丢入线程池执行
pool.shutdown(True) # 线程池任务全部执行完成后，主线程才会继续往下执行代码，类似join()

In [ ]:
# 线程池的回调函数：执行完线程池的方法后，可以调用另一个方法.类似后置处理teardown方法
list_a = [1,3,4]
list_b = ['a','b','c']
futures_list = []
for a in list_a:
    for b in  list_b:
        futures_2 = pool.submit(task_2,a,b)
        futures_2.add_done_callback(task_1) # 线程池任务执行完成后，调用task_1方法
        futures_list.append(futures_2)
        """注意：对于add_done_callback( task_1 )传入的函数名task_1,在定义ctask_1函数的时候一定要写一个位置参数,这里写的是feature，
        这个位置参数会通过add_done_callback( task_1 )方法自动传入, 而且这个参数就是 调用add_done_callback方法的feature_2对象"""
pool.shutdown(True)

for fu in futures_list:
    fu.result()

# 单例模式
对一个类进行实例化，即使多次实例化，也用同一个内存地址
如何实现呢？需要对类实例化时的过程进行判断,如果已经创建了实例化对象，直接返回，如果没有创建，就再调用基类的__new__()方法创建

In [ ]:
class Singleton(object):
    instance = None
    def __init__(self,name):
        self.name = name
    
    def __new__(cls,*args,**kwargs):
        if cls.instance:
            return cls.instance
        cls.instance = object.__new__(cls)
        return cls.instance

上面的单例模式的类看似简单实现了，但是当进行多线程对这个类实例化时，仍有可能单例模式失效，造成实例化出多个对象，所以改得加以改造，加上线程锁

In [ ]:

class Singleton2(object):
    cls_lock = threading.RLock
    instance = None
    def __new__(cls,*args,**kwargs):
        with cls.cls_lock:
            if cls.instance:
                return cls.instance
            cls.instance = object.__new__(cls)
            return cls.instance

多进程，主要学会multiprocessing.Process(target,args)方法的运用
创建多个进程时，也会默认在每个进程中创建1个线程
## 注意，对进程必须要在main函数下运行，这是受限于python的底层实现,不兼容spawn的方式创建进程
win仅支持spawn创建进程
Linux仅支持fork创建进程
mac支持fork和spawn，默认是spawn，可以通过如下方法更改

In [ ]:
import multiprocessing

multiprocessing.set_start_method('fork')

In [ ]:
import multiprocessing

processing_pool = []


def task(a, b):
    pass


if __name__ == '__main__':
    for i in range(10):
        process = multiprocessing.Process(target=task, args=(a, b))
        processing_pool.append(process)
        process.start()

    for i in processing_pool:
        i.join()

## 用python的多线程进行多并发请求测试某个接口，可行么？会不会受GIL锁的影响？
应该说影响不大，多并发请求是IO密集型操作，用python的threading模块多线程完全没问题
如果所做的操作是计算密集型，这个时候由于要用到计算机的多核性能，python的多线程受到GIL的限制，就不适用了，要用多进程。

# 常见的进程方法
p.start() 当前进程准备就绪，等待被CPU调度（工作单元是进程中的线程）
p.join()

## 守护进程

In [ ]:
process.daemon(True) #  设置为守护进程，主进程退出时，子进程也会退出
process.daemon(False) # 设置为守护进程，所有子进程全部结束，主进程才退出

## 设置进程name

In [ ]:
# 设置线程名字
process.name('my_process')
# 获取当前进程名字
multiprocessing.current_process().name
# 获取进程id
os.getpid()
# 获取进程的父进程id
os.getppid()
# 获取进程中有多少线程
len(threading.enumerate())

## 自定义进程类
类似自定义线程类，通过构建一个类继承multprocessing.Process,然后就可以在类中做定制化方法

In [ ]:
class MyProcessClass(multiprocessing.Process):
    def __init__(self):
        # 需要初始化的方法
        pass
    
    def run(self):
        # 需要定制使用的方法
        pass

# CPU个数
程序一般创建多个进程（利用CPU的多核优势），有多少核，创建多少进程

In [ ]:
# 获取cpu核个数
count = multiprocessing.cpu_count()
# 创建进程时要考虑主进程
for i in range(count-1):
    multiprocessing.Process(target=task_2,args=(a,b))

## 进程锁
类似线程锁的作用,进程之间用到数据共享时，需要用锁。

In [ ]:
manager = multiprocessing.Manager()
process_lock = manager.RLock() # 一定要用manager的递归锁
def multiprocess_func():
    with process_lock:
        # do something
        pass
process_pool = []
for i in range(10):
     p = multiprocessing.Process(target=multiprocess_func,args=(process_lock,)) # 必须把进程锁传入
     p.start()
     process_pool.append(p)

for i in process_pool:
    i.join()

## 进程池
限制进程创建的数量

In [ ]:
from concurrent.futures import ProcessPoolExecutor
p_pool = ProcessPoolExecutor(10)
pt = p_pool.submit(task_2,a,b) # 类似start()
pt.add_done_callback(task_1)
p_pool.shutdown(True) # 类似join()

# 协程
协程时程序员人员构造出的微线程技术，用户态内上下文切换技术，简而言之，是通过一个线程实现代码块互相切换执行
- asyncio模块的async语法